# Problem Statement

You have booked a reservation at a restaurant. At the time of booking, your position in the queue is `N + n`. You can observe the first `n` positions in the queue to estimate when you should leave home. Assume it takes `t` minutes to reach the restaurant. 

- If you arrive late, you will lose the reservation and the booking fee of `$c`.
- If you arrive early, you will incur a parking fee of `$h` per minute. 

Your goal is to determine the optimal time to leave home to minimizing your loses incurred from arriving late or arriving too early.

In [13]:
from utils import methods, math_expressions as me
import importlib
importlib.reload(methods)

import numpy as np
import math


# Explore Gamma Distribution

In [14]:
methods.plot_gamma(4, 2)

# Testing gamma parameters 

Method 1

In [15]:
alpha = 5
beta = 2
gamma_numbers = np.random.gamma(shape=alpha, scale=beta, size=10000)
e_mean = alpha * beta
e_std = math.sqrt(alpha) * beta
a_std = np.std(gamma_numbers)
a_mean = np.mean(gamma_numbers)
print(f'Expected Mean = {e_mean:.2f}, Actual Mean = {a_mean:.2f}')
print(f'Expected STD = {e_std:.2f}, Actual STD = {a_std:.2f}')

Expected Mean = 10.00, Actual Mean = 9.96
Expected STD = 4.47, Actual STD = 4.42


Method 2

In [16]:
alpha = 1/50
beta = 2
N = 10000
derived_gamma_numbers = [sum(np.random.gamma(shape=alpha, scale=beta, size=N)) for i in range(1000)]
e_mean = N * alpha * beta
e_std = math.sqrt(N * alpha) * beta
a_std = np.std(derived_gamma_numbers)
a_mean = np.mean(derived_gamma_numbers)
print(f'Expected Mean = {e_mean:.2f}, Actual Mean = {a_mean:.2f}')
print(f'Expected STD = {e_std:.2f}, Actual STD = {a_std:.2f}')



Expected Mean = 400.00, Actual Mean = 398.73
Expected STD = 28.28, Actual STD = 28.36


## Anaylise Hazard Function

In [17]:
alpha = 100
beta = 2
h = 5
c = 25
from scipy.stats import gamma
import plotly.graph_objects as go

u_star = methods.get_u_star_binary_fast(1, alpha, beta, h, c)

h_precise = lambda x: me.gamma_hazard_rate(alpha, beta).subs('x', x).evalf()
g_pdf = lambda x: gamma.pdf(x, alpha, scale=beta)
g_cdf = lambda x: gamma.cdf(x, alpha, scale=beta)
h_approx = lambda x: g_pdf(x) / (1 - g_cdf(x))
x_max = alpha*beta + 4 * math.sqrt(alpha) * beta


x = np.linspace(0, x_max * 3, 1000)
y_hazard = [float(h_precise(i)) for i in x]
y_hazard_approx = [h_approx(i) for i in x]
y_gamma = [g_pdf(i) for i in x]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_hazard, name='Hazrad Rate'))
fig.add_trace(go.Scatter(x=x, y=y_gamma, name='Gamma PDF'))
fig.add_trace(go.Scatter(x=x, y=y_hazard_approx, name='Approximated Hazard Rate'))
fig.add_trace(go.Scatter(x=[x_max], y=[float(h_precise(x_max))], mode='markers', marker=dict(size=10, color='red'), name=f'h(x_max={x_max:.2f}) = {float(h_precise(x_max)):.2f}'))
fig.add_vline(x=x_max, line_dash='dash', line_color='red', name=f'Mean + 4*STD={x_max:.2f}')
fig.add_trace(go.Scatter(x=[u_star], y=[float(h_precise(u_star))], mode='markers', marker=dict(size=10, color='green'), name=f'h(u*) = h/c = {float(h_precise(u_star)):.2f}'))
fig.add_vline(x=u_star, line_dash='dash', line_color='green', name=f'u*= {u_star:.2f}')
fig.show()

/var/folders/6b/tmvvscrj5wb5r4ngnndf1gtm0000gn/T/ipykernel_11277/2077782167.py:13: RuntimeWarning:

divide by zero encountered in scalar divide

